In [1]:
#default_exp modules.boxmatch

# Box Matching via IoU
> With this class we will match ground truth boxes to our calculated anchors with the intersection over union method.
> The threshold is defined in the config file.



In [2]:
import sys
sys.path.insert(0, "../pointpillars")
from pointpillars.utils.time import time_method
from pointpillars.utils.io import read_config

In [3]:
#export
import logging
import ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Union
from math import pi
from torchvision.ops import box_iou

from utils.math.box_ops import convert_boxes_to_2d_corners

logger = logging.getLogger(__name__)

In [4]:
#export
class BoxMatch(nn.Module):
    def __init__(self, pillars_cfg, head_cfg, training: bool = True):
        logger.info("Initializing BoxMatch module...")

        super(BoxMatch, self).__init__()
        self.pillars_cfg = pillars_cfg
        self.head_cfg = head_cfg
        self.anchors = ast.literal_eval(head_cfg['anchors'])
        self.training = training

        self.x_min =  self.pillars_cfg.getfloat('x_min')
        self.y_min =  self.pillars_cfg.getfloat('y_min')


        logger.debug("BoxMatch init complete.")

    def _calculate_absolute_boxes_from_anchors(self, pred_anchors: torch.Tensor) -> torch.Tensor:
        """Returns a view of all predicted, absolute bounding boxes.

        :param pred_anchors: tensor(batch_size, img_leng, img_with, nbr_anchors, nbr_attributes=7)

        :returns: tensor(batch_size, img_leng, img_width, nbr_anchors, nbr_attributes=7)
        """
        logger.info("Calculating absolute_boxes_from_anchors...")

        bs, n_x, n_y, n_a = pred_anchors.shape[0:4]
        x_step = (self.pillars_cfg.getfloat('x_max') - self.x_min) / n_x
        y_step = (self.pillars_cfg.getfloat('y_max') - self.y_min) / n_y
        # create tensor containing the pillar indicies in the correct dimension
        x = torch.cuda.FloatTensor(range(n_x)).unsqueeze(1).expand(n_x, n_y)
        y = torch.cuda.FloatTensor(range(n_y),).unsqueeze(0).expand(n_x, n_y)
        ind = torch.stack((x,y), dim=2).unsqueeze(0).expand(bs, -1, -1, -1)
        del x, y

        # add anchors to receive other absolute values
        anchors_tens = torch.cuda.FloatTensor(self.anchors)
        anchors_tens = anchors_tens.unsqueeze(0).unsqueeze(0).unsqueeze(0).expand(bs, n_x, n_y, -1, -1)

        # step with current pseudo image size
        # calculate pillar center from index
        pil_xy = ind * torch.cuda.FloatTensor([x_step, y_step])
        pil_xy += torch.cuda.FloatTensor([self.x_min, self.y_min])

        # expand for anchor nbr
        pil_xy = pil_xy.unsqueeze(3).expand(-1, -1, -1, n_a, -1)

        diag = torch.sqrt(torch.pow(pred_anchors[:,:,:,:,5], 2) + torch.pow(pred_anchors[:,:,:,:,4], 2))
        diag = diag.unsqueeze(4).expand(-1, -1, -1, -1, 2)
        # add center to offset
        pred_anchors_xy = pred_anchors[:,:,:,:,:2] * diag + pil_xy

        # add anchors z offset
        pred_anchors_z = pred_anchors[:,:,:,:,2] * pred_anchors[:,:,:,:,3] + anchors_tens[:,:,:,:,0]
        pred_anchors_z = pred_anchors_z.unsqueeze(4)
        pred_anchors_hwl = torch.exp(pred_anchors[:,:,:,:,3:6]) * anchors_tens[:,:,:,:,1:4]

        # add radiant offset
        epsilon = 1e-7
        pred_anchors_theta = torch.clamp(pred_anchors[:,:,:,:,6], -1 + epsilon, +1 - epsilon)
        pred_anchors_theta = -1 * torch.arcsin(pred_anchors_theta) + anchors_tens[:,:,:,:,4]
        pred_anchors_theta = pred_anchors_theta.unsqueeze(4)

        pred_anchors = torch.cat((pred_anchors_xy, pred_anchors_z, pred_anchors_hwl, pred_anchors_theta), dim=4)
        del anchors_tens, pred_anchors_z, pred_anchors_hwl, pred_anchors_theta

        logger.debug(f"Absolute boxes from anchors calculation complete.\n"
                     f"pred_anchors: {pred_anchors}{pred_anchors.shape}")

        return pred_anchors



    def _calculate_batch_iou(self, pred_boxes: torch.Tensor, gt_boxes: torch.Tensor) -> torch.Tensor:
        """Calculates the iou for batched prediction and ground truth boxes.

        :param pred_boxes: Tensor(batch_size, nbr_pred_boxes, 4)
                            with last dimension being (x_min, y_min, x_max, y_max)
        :param gt_boxes: Tensor(batch_size, nbr_gt_boxes, 4)
                            with last dimension being (x_min, y_min, x_max, y_max)

        :returns: Tensor(batch_size, nbr_pred_boxes, nbr_gt_boxes)
        """
        logger.info("Calculating batch iou...")
        logger.debug(f"pred_boxes: {pred_boxes}{pred_boxes.shape},\n"
                     f"gt_boxes: {gt_boxes}{gt_boxes.shape}")
        iou =  []
        for i in range(pred_boxes.shape[0]):
            pred_batch = pred_boxes[i]
            gt_batch = gt_boxes[i]
            iou.append(box_iou(pred_batch, gt_batch))

        batch_iou = torch.stack(iou, dim=0)
        logger.debug(f"Batch iou calculation complete.\n"
                     f"batch_iou: {batch_iou}{batch_iou.shape}")
        del pred_batch, gt_batch
        return batch_iou

    def boxmatch_from_corners(self,
                              preds: Union[list, tuple],
                              gt_boxes: torch.Tensor,
                              gt_mask: torch.Tensor,
                              pos_iou_threshold: float = 0.6,
                              neg_iou_threshold: float = 0.4) -> list:
        """
        Calculates iou from boxes in corner representation and uses non maximum suppression to get
        the best fitting boxes.

        :param preds: A tuple containing all predicted values for the batch. Must have the shape:
                      (pred_occ(N, H * W * nb_anchors),
                       pred_cls(N, H * W * nb_anchors),
                       pred_head(N, H * W * nb_anchors),
                       pred_box(N, H * W * nb_anchors, nb_attributes=7))

                       pred_box having attributes x,y,z,h,w,l,theta

        :param gt_boxes: Tensor(batch_size, nbr_gt_boxes, nbr_attributes=7)
        :param gt_mask: tensor(batch_size, nbr_gt_boxes)
                        Containing 1 or 0, depending if gt_box will be used for loss calculation
        :param pos_iou_threshold: float to give the upper threshold for positive box matching
        :param neg_iou_threshold: float to give the lower threshold for negative box matching
                                    only used to learn occupancy parameter

        :returns: A list containing the matched tensors. The list has the following structure:
                  [pred_occ(nb_matched_boxes),
                   pred_cls(nb_matched_boxes),
                   pred_head(nb_matched_boxes),
                   pred_boxes(nb_matched_boxes, nb_attributes=7),
                   gt_boxes(nb_matched_boxes, nb_attributes=7),
                   neg_matches(nb_negative_match)
                   gt_mask(nb_matched_boxes)]

                   neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold
        """
        logger.info("Calculation iou nms from corners...")
        logger.debug(f"preds: {preds}\n"
                     f"gt_boxes: {gt_boxes}{gt_boxes.shape}")

        if isinstance(preds, tuple):
            preds = list(preds)

        pred_boxes_corners = convert_boxes_to_2d_corners(preds[3])
        gt_boxes_corners = convert_boxes_to_2d_corners(gt_boxes)
        # upper iou threshold for positive as well as negative match
        # add x_min and y_min, since box_iou() expects values greater than 0
        bs, na, n = pred_boxes_corners.shape
        xy_min = torch.cuda.FloatTensor([self.x_min, self.y_min])
        xy_min = torch.cat((xy_min, xy_min), dim=0)
        iou = self._calculate_batch_iou(pred_boxes_corners + xy_min.expand_as(pred_boxes_corners),
                                        gt_boxes_corners + xy_min.expand_as(gt_boxes_corners))
        del pred_boxes_corners, gt_boxes_corners

        condition_pos = iou >= pos_iou_threshold
        # incorporate gt_mask
        gt_mask_pos = gt_mask.type(torch.BoolTensor).unsqueeze(1).expand(-1, condition_pos.shape[1], -1).cuda()
        condition_pos = torch.logical_and(condition_pos.cuda(), gt_mask_pos)
        indices_pos = torch.nonzero(condition_pos, as_tuple=False)
        del condition_pos, gt_mask_pos

        # batch_size * nbr_pred_boxes has to be added to index since pred_boxes is flattened
        indices_pred_boxes_flat = indices_pos[:,1] + indices_pos[:,0] * preds[3].shape[1]
        indices_gt_boxes_flat = indices_pos[:,2] + indices_pos[:,0] * gt_boxes.shape[1]
        del indices_pos

        results = []
        for tensor in preds:
            # flatten since batch dimension is no longer necessary after matching
            flat_tensor = torch.flatten(tensor, start_dim=0, end_dim=1)
            # select indices according to iou
            results.append(torch.index_select(flat_tensor, 0, indices_pred_boxes_flat))
        gt_boxes_flat = torch.flatten(gt_boxes, start_dim=0, end_dim=1)
        results.append(torch.index_select(gt_boxes_flat, 0, indices_gt_boxes_flat))
        del indices_pred_boxes_flat, flat_tensor, gt_boxes_flat, indices_gt_boxes_flat

        # negative matches
        # same index procedure as previously done
        condition_neg = iou <= neg_iou_threshold
        gt_mask_neg = gt_mask.type(torch.BoolTensor).unsqueeze(1).expand(-1, condition_neg.shape[1], -1).cuda()
        condition_neg = torch.logical_and(condition_neg, gt_mask_neg)
        indices_neg = torch.nonzero(condition_neg, as_tuple=False)
        indices_neg_flat = indices_neg[:,1] + indices_neg[:,0] * preds[3].shape[1]

        pred_occ_flat = torch.flatten(preds[0], start_dim=0, end_dim=1)
        results.append(torch.index_select(pred_occ_flat, 0, indices_neg_flat))
        del condition_neg, indices_neg, indices_neg_flat, pred_occ_flat, iou, gt_mask, gt_mask_neg

        logger.debug(f"iou nms from corners calc complete.\n"
                     f"results: {results}")

        return results

    def forward(self,
                preds: Union[list, tuple],
                gt_boxes: torch.Tensor,
                gt_mask: torch.Tensor) -> list:
        """Matches anchor predictions to gt_boxes with IoU

        :param preds: A tuple containing the predictions from the net. Must have the following shape:
                      (pred_occ(N, H, W, nb_anchors),
                       pred_cls(N, H, W, nb_anchors),
                       pred_head(N, H, W, nb_anchors),
                       pred_box(N, H, W, nb_anchors, nb_attributes=7))

        :param gt_boxes: tensor(batch_size, gt_boxes per image, nbr_attributes)
        :param gt_mask: tensor(batch_size, gt_boxes)
                        Containing 1 or 0, depending if gt_box will be used for loss calculation

        :returns: A list containing the matched tensors. The list has the following structure:
                  [pred_occ(nb_matched_boxes),
                   pred_cls(nb_matched_boxes),
                   pred_head(nb_matched_boxes),
                   pred_boxes(nb_matched_boxes, nb_attributes=7),
                   gt_boxes(nb_matched_boxes, nb_attributes=7),
                   neg_matches(nb_negative_match)
                   gt_mask(nb_matched_boxes)]

                   neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold
        """
        logger.info("Forward pass through BoxMatch...")
        logger.debug(f"preds: {preds},\n"
                     f"gt_boxes: {gt_boxes}{gt_boxes.shape}")

        if isinstance(preds, tuple):
            preds = list(preds)
        # calc anchor centers from pseudo image index
        preds[3] = self._calculate_absolute_boxes_from_anchors(preds[3])
        # shape Tensor(batch_size, H * W * nbr_anchors, nbr_attributes=7)
        for i in range(len(preds)):
            preds[i] = torch.flatten(preds[i], start_dim=1, end_dim=3)

        if self.training:
            return self.boxmatch_from_corners(preds,
                                              gt_boxes,
                                              gt_mask,
                                              pos_iou_threshold=self.pillars_cfg.getfloat('pos_iou_threshold'),
                                              neg_iou_threshold=self.pillars_cfg.getfloat('neg_iou_threshold'))
        else:
            return preds


In [5]:
pillars_cfg = read_config()['pillars']
head_cfg = read_config()['head']
tens = torch.rand([2, 252, 252, 4, 7]).cuda()
boxmatch = BoxMatch(pillars_cfg, head_cfg)
tens = boxmatch._calculate_absolute_boxes_from_anchors(tens)

tens, tens.shape

(tensor([[[[[5.2858e-01, 3.9131e-01, 3.9089e+00,  ..., 1.9244e+00,
             6.4540e-01, 3.4309e-01],
            [1.1375e+00, 7.4151e-01, 4.0299e+00,  ..., 1.7846e+00,
             4.6101e-01, 1.6108e+00],
            [1.1225e+00, 7.1058e-01, 1.6348e+00,  ..., 2.0685e+00,
             8.6185e-01, 3.4559e-01],
            [3.3530e-01, 5.8963e-01, 1.0287e+00,  ..., 1.9457e+00,
             6.1868e-01, 1.7314e+00]],
 
           [[1.1104e+00, 5.8405e-01, 4.6674e+00,  ..., 2.5362e+00,
             3.4695e-01, 6.5566e-01],
            [6.1030e-01, 1.0508e+00, 4.5466e+00,  ..., 2.2604e+00,
             4.4306e-01, 2.1830e+00],
            [5.2022e-01, 1.0885e+00, 1.4813e+00,  ..., 2.1505e+00,
             8.6905e-01, 9.9792e-01],
            [8.8797e-01, 7.2414e-01, 1.0687e+00,  ..., 2.3338e+00,
             4.7345e-01, 2.0060e+00]],
 
           [[4.6342e-01, 1.5703e+00, 4.0111e+00,  ..., 2.3703e+00,
             4.2947e-01, 7.8445e-01],
            [2.0504e-01, 1.0849e+00, 4.6331e+00, 

In [6]:
tens = boxmatch._convert_boxes_to_corners(tens)
tens, tens.shape


AttributeError: 'BoxMatch' object has no attribute '_convert_boxes_to_corners'

In [ ]:
pred_boxes = torch.rand([2, 254016, 4]).cuda()
gt_boxes = torch.rand([2, 2, 4]).cuda()
boxmatch._calculate_batch_iou(pred_boxes, gt_boxes)

In [ ]:
time_method(boxmatch._calculate_batch_iou, runs=100, kwargs={'pred_boxes': pred_boxes, 'gt_boxes': gt_boxes})

In [ ]:
pred_boxes = torch.cuda.FloatTensor([[[0, 0, 2, 2], [0, 1, 2, 2], [0, 1, 2, 3]], [[1, 1, 2, 2], [0, 0, 0, 0], [0, 0, 0, 0]]])
gt_boxes = torch.cuda.FloatTensor([[[0, 0, 2, 2], [1, 1, 2, 2]], [[0, 0, 2, 2], [1, 1, 2, 2]]])

out = boxmatch._calculate_batch_iou(pred_boxes, gt_boxes)
out, out.shape

In [ ]:
pred_boxes = torch.cuda.FloatTensor([[[0,0,0,1,1,1,0], [0,0,0,2,2,2,0], [0,0,0,1,2,1,0], [0,0,0,0,0,0,0]]])
pred_cls = torch.cuda.FloatTensor([[0.1, 0.2, 0.3, 0.4]])
pred_heading = torch.cuda.FloatTensor([[0, 1, 0, 0]])
pred_occ = torch.cuda.FloatTensor([[0.1, 0.2, 0.3, 0.4]])
gt_boxes = torch.cuda.FloatTensor([[[0,0,0,1,1,1,0], [0,0,0,2,2,2,0]]])
gt_mask = torch.cuda.FloatTensor([[[0,1]]])

preds = [pred_occ, pred_cls, pred_heading, pred_boxes]

out = boxmatch.iou_nms_from_corners(preds, gt_boxes, gt_mask)
out[0], out[0].shape, out[1], out[1].shape, out[2], out[2].shape, out[3], out[3].shape, out[4], out[4].shape, out[5], out[5].shape
